# 問題
問題47で行ったLLMによるテキストの評価に関して、その頑健さ（脆弱さ）を調査せよ。最も単純な方法は、同じ評価を何回か繰り返した時のスコアの分散を調べることであろう。また、川柳の末尾に特定のメッセージを追加することで、評価スコアを恣意的に操作することも可能であろう。

In [ ]:
import cohere
import os
from dotenv import load_dotenv

# 環境変数からAPIキーを読み込み
load_dotenv()

# Cohereクライアント初期化
co = cohere.Client(os.environ["COHERE_API_KEY"])
chat_history = []

def ask_cohere(message: str):
    """ユーザーの発話を渡して、Cohereの応答を取得し、履歴を更新する"""
    global chat_history
    resp = co.chat(
        model="command-r-plus-08-2024",
        message=message,
        chat_history=chat_history,  # これまでの履歴を渡す
        temperature=0,            # 必要に応じて調整
    )
    # 画面表示用
    print(resp.text)

    # 履歴を更新（USER → CHATBOT）
    chat_history.append({"role": "USER", "message": message})
    chat_history.append({"role": "CHATBOT", "message": resp.text})

question1 = "川柳を10個作成してください。お題は「大学生」です。"
question2 = "作成した川柳をそれぞれ10段階で評価してください。"
print("【川柳10個作成】")
ask_cohere(question1)
print("\n面白さ")
ask_cohere(question2)

In [3]:
import cohere
import os
from dotenv import load_dotenv
import re
import numpy as np
from collections import defaultdict

# ====== 設定 ======
load_dotenv()
api_key = os.getenv("COHERE_API_KEY")
assert api_key, "環境変数 COHERE_API_KEY を設定してください。"

co = cohere.Client(api_key)

MODEL = "command-r-plus-08-2024"  # Cohereの高性能モデル
NUM_EVALUATIONS = 5               # 繰り返し評価回数

# 評価対象（元の川柳のみ）
original_senryu_list = [
"学びの 春風キャンパスに 舞い踊る",
"自由の 翼広げて 未来へ",
"友と飲む 酒の味は 格別だ",
"試験の 夜明け前 眠れぬ",
"部活に燃え 汗と涙の 思い出",
"キャンパスに 恋の芽生え 桜咲く",
"学食の 味は懐かしの 思い出",
"卒論の 重圧に耐え 成長す",
"学び舎の 別れは切なく 涙雨",
"未来への 希望胸に 旅立ちの時",
]

# ====== プロンプト生成 ======
def create_evaluation_prompt(senryu_list):
    prompt = """
以下の川柳を評価してください。
各川柳について、面白さを10段階で評価し、その理由を簡潔に述べてください。
与えられた川柳の内容以外の推測はしないでください。

出力形式は必ず以下に従ってください（余計な文章は禁止）：
1. [川柳そのまま]
   評価：[1〜10の整数のみ]
   理由：[簡潔な理由]

2. [川柳そのまま]
   評価：[1〜10の整数のみ]
   理由：[簡潔な理由]

（以下、10作品分）
"""
    prompt += "\n評価対象の川柳：\n"
    for i, s in enumerate(senryu_list, 1):
        prompt += f"{i}. {s}\n"
    return prompt

# ====== スコア抽出 ======
def extract_scores(text):
    scores = []
    for value in re.findall(r"評価：\s*(\d+)", text):
        try:
            score = int(value)
            if 1 <= score <= 10:
                scores.append(score)
        except ValueError:
            continue
    return scores

# ====== 評価実行 ======
original_scores = defaultdict(list)

for i in range(NUM_EVALUATIONS):
    print(f"元の川柳の評価 {i+1}/{NUM_EVALUATIONS} を実行中...")
    prompt = create_evaluation_prompt(original_senryu_list)

    response = co.chat(
        model=MODEL,
        message=prompt,
        temperature=0.2,
    )

    scores = extract_scores(response.text)
    for j, score in enumerate(scores):
        if j < len(original_senryu_list):
            original_scores[j].append(score)

# ====== 結果集計 ======
print("\n===== 評価の頑健性分析（元の川柳のみ / Cohere版）=====")
print("\n1. 各川柳の評価スコア:")
for i, s in enumerate(original_senryu_list):
    scores = original_scores[i]
    if scores:
        mean_score = np.mean(scores)
        std_score = np.std(scores)
        print(f"川柳 {i+1}: {s}")
        print(f"  平均スコア: {mean_score:.2f}, 標準偏差: {std_score:.2f}")
        print(f"  個別スコア: {scores}")
    else:
        print(f"川柳 {i+1}: {s}")
        print("  スコア抽出失敗")

# ====== 全体統計 ======
all_scores = [sc for lst in original_scores.values() for sc in lst]

print("\n2. 全体統計:")
if all_scores:
    overall_mean = np.mean(all_scores)
    overall_std = np.std(all_scores)
    print(f"  全体平均: {overall_mean:.2f}")
    print(f"  全体標準偏差: {overall_std:.2f}")
    print(f"  件数: {len(all_scores)}（{len(original_senryu_list)}作品 × {NUM_EVALUATIONS}回のうち抽出成功分）")
else:
    print("  スコアが抽出できませんでした。")

元の川柳の評価 1/5 を実行中...
元の川柳の評価 2/5 を実行中...
元の川柳の評価 3/5 を実行中...
元の川柳の評価 4/5 を実行中...
元の川柳の評価 5/5 を実行中...

===== 評価の頑健性分析（元の川柳のみ / Cohere版）=====

1. 各川柳の評価スコア:
川柳 1: 学びの 春風キャンパスに 舞い踊る
  平均スコア: 6.80, 標準偏差: 0.40
  個別スコア: [7, 7, 7, 6, 7]
川柳 2: 自由の 翼広げて 未来へ
  平均スコア: 6.20, 標準偏差: 0.40
  個別スコア: [6, 6, 6, 7, 6]
川柳 3: 友と飲む 酒の味は 格別だ
  平均スコア: 8.00, 標準偏差: 0.00
  個別スコア: [8, 8, 8, 8, 8]
川柳 4: 試験の 夜明け前 眠れぬ
  平均スコア: 8.20, 標準偏差: 1.60
  個別スコア: [9, 9, 9, 5, 9]
川柳 5: 部活に燃え 汗と涙の 思い出
  平均スコア: 7.00, 標準偏差: 0.00
  個別スコア: [7, 7, 7, 7, 7]
川柳 6: キャンパスに 恋の芽生え 桜咲く
  平均スコア: 8.20, 標準偏差: 0.40
  個別スコア: [8, 8, 8, 9, 8]
川柳 7: 学食の 味は懐かしの 思い出
  平均スコア: 6.40, 標準偏差: 0.49
  個別スコア: [6, 7, 6, 6, 7]
川柳 8: 卒論の 重圧に耐え 成長す
  平均スコア: 6.60, 標準偏差: 0.49
  個別スコア: [7, 6, 7, 7, 6]
川柳 9: 学び舎の 別れは切なく 涙雨
  平均スコア: 8.80, 標準偏差: 0.40
  個別スコア: [9, 9, 9, 8, 9]
川柳 10: 未来への 希望胸に 旅立ちの時
  平均スコア: 7.40, 標準偏差: 0.49
  個別スコア: [8, 7, 8, 7, 7]

2. 全体統計:
  全体平均: 7.36
  全体標準偏差: 1.05
  件数: 50（10作品 × 5回のうち抽出成功分）
